In [12]:
import geopandas as gpd
import pandas as pd
import locale
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from IPython.display import display, Markdown
pd.options.display.float_format = '{:.4f}'.format
pd.set_option('display.precision', 4)
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')
import warnings
warnings.filterwarnings('ignore')
glebas = gpd.read_file('../glebas-federais.gpkg', layer='glebas-mais-amazonia')


In [13]:
mun = gpd.read_file('../glebas-federais.gpkg', layer='municipios-area-estudo', 
                    include_fields=['nm_regiao', 'cd_uf', 'nm_uf', 'sigla', 
                                    'cd_mun', 'nm_mun'])
glebas = gpd.read_file('../glebas-federais.gpkg', layer='glebas-mais-amazonia', 
                       include_fields=['gid','nome_gleba', 'area_ha', 'uf_nome'])
glebas = glebas.set_index('gid')

## Abrangência Municipal

Criar geopackge com uma tabela pra cada gleba contendo os municípios que se interceptam com ela.

In [ ]:
for row in glebas.itertuples():
    indice = int(row.Index)
    gleba = glebas.loc[[indice]]
    print(f' indice {indice}')
    gleba = gleba.rename(columns={'nome_gleba':'Nome da Gleba',
    'area_ha':'Área (ha)'})
    municipio_gleba = mun.sjoin(gleba, how='inner')
    municipio_gleba = municipio_gleba.rename(columns={'cd_uf':'Código da UF',
    'nm_uf':'Estado', 'sigla':'UF', 'cd_mun':'Código do Município',
    'nm_mun':'Nome do Município'})
    gpd.GeoDataFrame(municipio_gleba).to_file('../municipios-glebas.gpkg', 
                                              layer=f'{indice}',
                                              driver="GPKG")

## Unidades de Conservação

In [ ]:
for row in glebas.itertuples():
    indice = int(row.Index)
    gleba = glebas.loc[[indice]]
    print(f' indice {indice}')
    uc=gpd.read_file('../glebas-federais.gpkg', layer='uc', mask=gleba)
    uc_gleba = uc.overlay(gleba, how='intersection')
    if uc_gleba.shape[0]>0:
        uc_gleba['Área Sobreposta (ha)'] = (uc_gleba.to_crs(5880).area)/10000
        uc_gleba = uc_gleba.rename(columns={'nome_uc1':'Nome',
        'categori3':'Categoria','esfera5':'Responsabilidade'})
        gpd.GeoDataFrame(uc_gleba).to_file('../uc-glebas.gpkg', 
                                              layer=f'{indice}',
                                              driver="GPKG")

## Terra Indígenas

In [ ]:
for row in glebas.itertuples():
    indice = int(row.Index)
    gleba = glebas.loc[[indice]]
    print(f' indice {indice}')
    ti=gpd.read_file('../glebas-federais.gpkg', layer='ti-portarias',
                    mask=gleba)
    ti = ti.set_crs(4674, allow_override=True)
    ti_gleba = ti.overlay(gleba, how='intersection')
    if ti_gleba.shape[0]>0:
        ti_gleba['Área Sobreposta (ha)'] = (ti_gleba.to_crs(5880).area)/10000
        ti_gleba = ti_gleba.rename(columns={'terrai_nom':'Nome',
                                        'fase_ti':'Fase de Regularização'})
        gpd.GeoDataFrame(ti_gleba).to_file('../ti-glebas.gpkg', 
                                              layer=f'{indice}',
                                              driver="GPKG")

## Projetos de Assentamento

In [ ]:

for row in glebas.itertuples():
    indice = int(row.Index)
    gleba = glebas.loc[[indice]]
    print(f' indice {indice}')
    pa = gpd.read_file('../glebas-federais.gpkg', layer='pa_brasil',
                        mask=gleba)
    pa_gleba = pa.overlay(gleba, how='intersection')
    if pa_gleba.shape[0] > 0:
        pa_gleba['Área Sobreposta (ha)'] = (pa_gleba.to_crs(5880).area)/10000
        pa_gleba = pa_gleba.rename(columns={'cd_sipra':'SIPRA',
                                'nome_proje':'Nome','municipio':'Município'})
        gpd.GeoDataFrame(pa_gleba).to_file('../pa-glebas.gpkg', 
                                              layer=f'{indice}',
                                              driver="GPKG")

## Território Quilombola

In [ ]:
for row in glebas.itertuples():
    indice = int(row.Index)
    gleba = glebas.loc[[indice]]
    print(f' indice {indice}')
    tq = gpd.read_file('../glebas-federais.gpkg',
                        layer='tq_brasil', mask=gleba)
    tq_gleba = tq.overlay(gleba, how='intersection')
    if tq_gleba.shape[0] >0:
        tq_gleba['Área Sobreposta (ha)'] = (tq_gleba.to_crs(5880).area)/10000
        tq_gleba = tq_gleba.rename(columns={'nm_comunid':'Nome',
                'esfera':'Responsabilidade', 'fase':'Fase da Regularização'})
        gpd.GeoDataFrame(tq_gleba).to_file('../tq-glebas.gpkg', 
                                              layer=f'{indice}',
                                              driver="GPKG")

## SIGEF

In [ ]:
for row in glebas.itertuples():
    indice = int(row.Index)
    gleba = glebas.loc[[indice]]
    sigef = gpd.read_file('../glebas-federais.gpkg', layer='sigef-particular', mask=gleba)
    print(f' indice {indice}')
    sigef_gleba = sigef.overlay(gleba, how='intersection')
    if sigef_gleba.shape[0] > 0:
        sigef_gleba['Área Sobreposta (ha)'] = (sigef_gleba.to_crs(5880).area)/10000
        sigef_gleba = sigef_gleba.rename(columns={'parcela_co':'Código SIGEF','natureza':'Natureza do Polígono'})
        gpd.GeoDataFrame(sigef_gleba).to_file('../sigef-glebas.gpkg', 
                                              layer=f'{indice}',
                                              driver="GPKG")

## SNCI

In [ ]:
for row in glebas.itertuples():
    indice = int(row.Index)
    gleba = glebas.loc[[indice]]
    snci = gpd.read_file('../glebas-federais.gpkg', layer='snci-particular', mask=gleba)
    print(f' indice {indice}')
    snci_gleba = snci.overlay(gleba, how='intersection')
    if snci_gleba.shape[0] > 0:
        snci_gleba['Área Sobreposta (ha)'] = (snci_gleba.to_crs(5880).area)/10000
        snci_gleba = snci_gleba.rename(columns={'num_certif':'Código SNCI'})
        gpd.GeoDataFrame(snci_gleba).to_file('../snci-glebas.gpkg', 
                                              layer=f'{indice}',
                                              driver="GPKG")